# Name: Srinibas Masanta
# Roll Number: 202318054

# Real Estate California

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [14]:
data = pd.read_csv('/content/RealEstate_California.csv') #loading dataset

In [15]:
data.head()

,Unnamed: 0,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County


In [16]:
data = data.drop(columns=['Unnamed: 0', 'id', 'datePostedString', 'streetAddress', 'description', 'currency'])  #dropping irrelevant columns
data = data.dropna()  #dropping missing values

In [17]:
# Initialize label encoders dictionary
label_encoders = {}
categorical_columns = data.select_dtypes(include=['object']).columns

# Use .loc[] to avoid SettingWithCopyWarning
for column in categorical_columns:
    le = LabelEncoder()
    data.loc[:, column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le

In [21]:
X = data.drop(columns=['price'])  #features
y = data['price']                 #target

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting

In [23]:
#Normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Tensorflow

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [38]:
# Model definition with regularization and dropout
model_tf = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),                # Input layer
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)),  # L2 regularization on the first layer
    layers.Dropout(0.2),                                     # Dropout for regularization
    layers.Dense(32, activation='relu',
                 kernel_regularizer=regularizers.l2(0.01)),  # L2 regularization on the second layer
    layers.Dense(1)                                          # Output layer
])

In [41]:
# Compile the model
model_tf.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history_tf = model_tf.fit(X_train, y_train, epochs=300, validation_split=0.2, batch_size=32)

Epoch 1/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 4949094170624.0000 - mae: 900865.4375 - val_loss: 2085939314688.0000 - val_mae: 741274.5625
Epoch 2/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2937724076032.0000 - mae: 830052.1875 - val_loss: 2077272834048.0000 - val_mae: 738720.6875
Epoch 3/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3309135200256.0000 - mae: 875277.3125 - val_loss: 2068574896128.0000 - val_mae: 736216.3125
Epoch 4/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6085397184512.0000 - mae: 975855.3750 - val_loss: 2058754850816.0000 - val_mae: 733423.3750
Epoch 5/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4075914526720.0000 - mae: 846306.5000 - val_loss: 2050077360128.0000 - val_mae: 730914.9375
Epoch 6/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3489082638336.0000 - mae: 819561.5000 - val_loss: 2041650610176.0000 - val_mae: 728456.8750
Epoch 7/300
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5097022029824.0000 - mae: 925866.7

In [42]:
# Model evaluation
test_loss_tf, test_mae_tf = model_tf.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss_tf}, Test MAE: {test_mae_tf}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1623366565888.0000 - mae: 701783.0000 
Test Loss: 2385957093376.0, Test MAE: 741466.6875


### PyTorch

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim

In [44]:
# Model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)   # Simplified to fewer units
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)                  # Output layer
        self.dropout = nn.Dropout(0.2)               # Dropout for regularization
        self.relu = nn.ReLU()                        # ReLU activation

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # Final output
        return x

In [45]:
model_pt = ANNModel()    #Instance

In [46]:
# Loss function and optimizer
criterion = nn.MSELoss()  # Mean squared error for regression
optimizer = optim.Adam(model_pt.parameters(), lr=0.001, weight_decay=0.01)  # L2 regularization through weight_decay

In [47]:
# Converting to PyTorch tensors
X_train_pt = torch.tensor(X_train, dtype=torch.float32)
y_train_pt = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_pt = torch.tensor(X_test, dtype=torch.float32)
y_test_pt = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [48]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model_pt.train()
    optimizer.zero_grad()
    outputs = model_pt(X_train_pt)
    loss = criterion(outputs, y_train_pt)
    loss.backward()
    optimizer.step()

In [49]:
# Evaluation
model_pt.eval()
with torch.no_grad():
    y_pred_pt = model_pt(X_test_pt)
    test_loss_pt = criterion(y_pred_pt, y_test_pt).item()

In [50]:
# MAE calculation
mae_pt = torch.mean(torch.abs(y_pred_pt - y_test_pt)).item()

print(f"TensorFlow Test MAE: {test_mae_tf:.4f}")
print(f"PyTorch Test MAE: {mae_pt:.4f}")

TensorFlow Test MAE: 741466.6875
PyTorch Test MAE: 1182907.3750


# Csgo round snapshots

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
data2 = pd.read_csv('/content/csgo_round_snapshots.csv')

In [89]:
data2.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [ ]:
#Encoding categorical variables
label_encoders = {}
categorical_cols = ['map', 'bomb_planted', 'round_winner']
for col in categorical_cols:
    le = LabelEncoder()
    data2[col] = le.fit_transform(data2[col])
    label_encoders[col] = le

In [ ]:
#Normalizing numerical features
scaler = StandardScaler()
numerical_cols = data2.columns.drop(categorical_cols)
data2[numerical_cols] = scaler.fit_transform(data2[numerical_cols])

In [ ]:
X = data2.drop('round_winner', axis=1) #features
y = data2['round_winner']              #target

In [ ]:
y = to_categorical(y, num_classes=2)   #onehot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  #splitting

In [ ]:
#Model
model_tf = Sequential()
model_tf.add(Input(shape=(X_train.shape[1],)))
model_tf.add(Dense(128, activation='relu'))
model_tf.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01)),)
model_tf.add(Dense(32, activation='relu'))
model_tf.add(Dense(2, activation='softmax'))

In [ ]:
model_tf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_tf.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7541 - loss: 0.4600 - val_accuracy: 0.7568 - val_loss: 0.4520
Epoch 2/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7569 - loss: 0.4492 - val_accuracy: 0.7610 - val_loss: 0.4495
Epoch 3/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7635 - loss: 0.4405 - val_accuracy: 0.7643 - val_loss: 0.4418
Epoch 4/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7709 - loss: 0.4323 - val_accuracy: 0.7652 - val_loss: 0.4503
Epoch 5/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7711 - loss: 0.4300 - val_accuracy: 0.7650 - val_loss: 0.4427
Epoch 6/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7760 - loss: 0.4235 - val_accuracy: 0.7660 - val_loss: 0.4335
Epoch 7/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.7786 - loss: 0.4189 - val_accuracy: 0.7675 - val_loss: 0.4410
Epoch 8/10
3061/3061 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.7792 - loss: 0.4172 

In [ ]:
loss, accuracy = model_tf.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7749 - loss: 0.4307


In [ ]:
class CSGOModel(nn.Module):
    def __init__(self):
        super(CSGOModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x

In [ ]:
#Converting data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
model_pt = CSGOModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001, weight_decay=0.01)

In [ ]:
#Loop
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model_pt(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.6927
Epoch [2/10], Loss: 0.6890
Epoch [3/10], Loss: 0.6854
Epoch [4/10], Loss: 0.6818
Epoch [5/10], Loss: 0.6780
Epoch [6/10], Loss: 0.6739
Epoch [7/10], Loss: 0.6697
Epoch [8/10], Loss: 0.6650
Epoch [9/10], Loss: 0.6601
Epoch [10/10], Loss: 0.6547


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
with torch.no_grad():
    outputs = model_pt(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    y_test_labels = torch.max(y_test_tensor, 1)[1]
    accuracy = accuracy_score(y_test_labels, predicted)

In [ ]:
print(f'TensorFlow Model Accuracy: {accuracy:.4f}')
print(f'PyTorch Model Accuracy: {accuracy:.4f}')

TensorFlow Model Accuracy: 0.7156
PyTorch Model Accuracy: 0.7156
